In [ ]:
Thư viện **File System** (fs) có 3 cách sử dụng:
- Đọc/ghi file Bất đồng bộ:
	- Cách 1: Thông qua **Promise** (Những thao tác đọc file, ghi file thì sẽ trả ra Promise) - Sử dụng `.then`, `.catch` để lấy kết quả hoặc xử lý lỗi
	- Cách 2: Thông qua **Callback** (Cách truyền thống) để trả về kết quả hoặc lỗi
- Cách 3:  Đọc/ghi file đồng bộ

---

**Promise API**:
- `filehandle.readFile(options)`: Đọc nội dung của 1 file
- `filehandle.writeFile(data, options)`: Ghi nội dung của 1 file
- `fsPromises.copyFile(src, dest[, mode])`: Copy file từ ... sang ...
- `fsPromises.cp(src, dest[, mode])`: Copy file từ ... sang ...
- `fsPromises.mkdir(path[, options])`: Tạo thư mục
- `fsPromises.readdir(path[, options])`: Đọc thư mục xem bên trong chứa những gì
- `fsPromises.rename(oldPath, newPath)`: Đổi tên file hoặc tên thư mục
- `fsPromises.rm(path[, options])`: Xoá thư mục hoặc xoá file
- `fsPromises.symlink(target, path[, type])`: Tạo lối tắt (Giống Sent to Shortcut trong window)
- `fsPromises.unlink(path)`: Cũng dùng để xoá
- `fsPromises.watch(filename[, options])`: Theo dõi một file khi nào bị sửa (Code ra Live Server)
-...

**Callback**:
- Cũng tương tự tên: `readFile`, `writeFile`...
- Tham số có thêm callback. VD: `fs.appendFile(path, data[, options], callback)` // Thêm vào cuối file

**Sync**:
- Tên tương tự như thêm đuôi `"Sync"`. VD: `readFileSync`, `mkdirSync`...

In [ ]:
// Vấn đề tại file ./server.js

/* In memory DB */
const db = {
    tasks: [
        {
            id: 1,
            title: "Nấu cơm",
        },
        {
            id: 2,
            title: "Quét nhà",
        },
    ],
};

// Vấn đề: Khi ta lưu `db` ở In memory, và khi restart server là database sẽ bị reset lại từ đầu
// Giải pháp: Nếu chúng ta đọc/ghi file và lưu database ra một file thì sẽ khắc phục được lỗi đó

In [ ]:
const http = require("node:http");

/* Cú pháp khởi tạo */
const fs = require("node:fs/promises"); // /promises: sử dụng với Promise
// const fs = require("node:fs") // sử dụng với callback hoặc sync


In [ ]:
/**
 * Ghi file
 */
const http = require("node:http");
const fs = require("node:fs/promises");

/* In memory DB */
const db = {
    tasks: [
        {
            id: 1,
            title: "Nấu cơm",
        },
        {
            id: 2,
            title: "Quét nhà",
        },
    ],
};

/* Ghi file */
// 1. Đường dẫn của file
const filePath = "./db.json";

// 2. Ghi file: Dữ liệu nhận vào phải là dạng chuỗi
// Mỗi lần ghi file là sẽ GHI ĐÈ TOÀN BỘ lên nội dung cũ
// writeFile: ghi đè toàn bộ
// append: Chèn xuống dòng cuối --> nhưng có thể gây lỗi vì sai định dạng
fs.writeFile(filePath, JSON.stringify(db, null, 4), "utf-8");


In [ ]:
/* Đọc file: Kết quả Dạng chuỗi => cần JSON.parse */
fs.readFile(filePath, "utf-8")
    .then((result) => {
        console.log(JSON.parse(result));
    })
    // Bắt lỗi nếu file không tồn tại!
    .catch((error) => {
        if (error.code === "ENOENT") {
            // Xử lý tạo file mới nếu lỗi là file không tồn tại
        }
    });

// Cần xử lý nếu file không tồn tại:
// - Nếu file không tồn tại: Thực hiện ghi file -> sẽ tạo file mới đấy
// - Nếu file không tồn tại: Đọc file sẽ lỗi

In [ ]:
// File hoàn chỉnh

const http = require("node:http");
const fs = require("node:fs/promises");

/* In memory DB */
const db = {
    tasks: [
        {
            id: 1,
            title: "Nấu cơm",
        },
        {
            id: 2,
            title: "Quét nhà",
        },
    ],
};

/* Ghi file */
const filePath = "./db.json";
// fs.writeFile(filePath, JSON.stringify(db, null, 4), "utf-8");

/* Đọc file: Kết quả Dạng chuỗi => cần JSON.parse */
fs.readFile(filePath, "utf-8")
    .then((result) => {
        console.log(JSON.parse(result));
    })
    // Bắt lỗi nếu file không tồn tại!
    .catch((error) => {
        if (error.code === "ENOENT") {
            // Xử lý tạo file mới nếu lỗi là file không tồn tại
        }
    });


In [ ]:
/**
 * Tạo utils: Đọc/ghi file
 */
const fs = require("node:fs/promises");
const filePath = "./db.json";

/* Đọc file */
async function read() {
    try {
        const result = await fs.readFile(filePath, "utf-8");
        return JSON.parse(result);
    } catch (error) {
        if (error.code === "ENOENT") {
            // todo: Ghi file: {}
            // Nếu file db.json thì chúng ta tạo file đó.
            // Nội dung trong file db.json mặc định là {} rỗng.
            const defaultDB = {};
            save(defaultDB); // Mặc định
            return defaultDB;
        }
    }
}

/* Ghi file */
async function save(db) {
    try {
        await fs.writeFile(filePath, JSON.stringify(db, null, 4), "utf-8");
    } catch (error) {
        // Lỗi liên quan đến hệ thống
        console.log(error);
    }
}

/* Export một object có 2 hàm: read và save*/
module.exports = { read, save };
